In [6]:
import sys
sys.path.append('../../src')
from analytics.simple_analytics import TraceAnalysis
import pandas as pd

# Load the latest results
ta = TraceAnalysis('results/20250622_180858')
print(ta)

TraceAnalysis(path='results/20250622_180858', traces=2473774)


# Keltner Strategy Analysis - Latest Results

Analysis of the most recent Keltner strategy backtest results.

# Keltner Strategy Analysis - Latest Results

Analysis of the most recent Keltner strategy backtest results.

# Keltner Strategy Analysis - Latest Results

Analysis of the most recent Keltner strategy backtest results.

In [ ]:
# Signal distribution
signals = ta.sql('''
    SELECT 
        signal_value,
        COUNT(*) as count,
        COUNT(DISTINCT strategy_id) as strategies_using
    FROM traces
    GROUP BY signal_value
    ORDER BY signal_value
''')
signals

In [ ]:
# Signal distribution
signals = ta.sql('''
    SELECT 
        signal_value,
        COUNT(*) as count,
        COUNT(DISTINCT strategy_id) as strategies_using
    FROM traces
    GROUP BY signal_value
    ORDER BY signal_value
''')
signals

In [ ]:
# Signal distribution
signals = ta.sql('''
    SELECT 
        signal_value,
        COUNT(*) as count,
        COUNT(DISTINCT strategy_id) as strategies_using
    FROM traces
    GROUP BY signal_value
    ORDER BY signal_value
''')
signals

## Strategy Activity Analysis

In [ ]:
# Signal distribution
signals = ta.sql('''
    SELECT 
        signal_value,
        COUNT(*) as count,
        COUNT(DISTINCT strategy_id) as strategies_using
    FROM traces
    GROUP BY signal_value
    ORDER BY signal_value
''')
signals

In [ ]:
# Most active strategies
active = ta.sql('''
    SELECT 
        strategy_id,
        COUNT(*) as signal_changes,
        MIN(bar_idx) as first_signal,
        MAX(bar_idx) as last_signal,
        COUNT(DISTINCT signal_value) as unique_signals
    FROM traces
    GROUP BY strategy_id
    ORDER BY signal_changes DESC
    LIMIT 10
''')
print("Most Active Strategies:")
active

In [ ]:
# Least active strategies
inactive = ta.sql('''
    SELECT 
        strategy_id,
        COUNT(*) as signal_changes
    FROM traces
    GROUP BY strategy_id
    ORDER BY signal_changes ASC
    LIMIT 10
''')
print("Least Active Strategies:")
inactive

## Trade Performance Analysis

In [ ]:
# Calculate performance for all strategies
performance = ta.sql('''
    WITH signal_changes AS (
        SELECT 
            strategy_id,
            bar_idx,
            signal_value,
            price,
            LAG(signal_value) OVER (PARTITION BY strategy_id ORDER BY bar_idx) as prev_signal
        FROM traces
    ),
    trades AS (
        SELECT 
            strategy_id,
            bar_idx as entry_bar,
            price as entry_price,
            signal_value as direction,
            LEAD(bar_idx) OVER (PARTITION BY strategy_id ORDER BY bar_idx) as exit_bar,
            LEAD(price) OVER (PARTITION BY strategy_id ORDER BY bar_idx) as exit_price
        FROM signal_changes
        WHERE signal_value != 0 AND (prev_signal = 0 OR prev_signal IS NULL)
    ),
    strategy_performance AS (
        SELECT 
            strategy_id,
            COUNT(*) as num_trades,
            AVG((exit_price - entry_price) / entry_price * direction * 100) as avg_return_pct,
            SUM((exit_price - entry_price) / entry_price * direction * 100) as total_return_pct,
            SUM(CASE WHEN (exit_price - entry_price) * direction > 0 THEN 1 ELSE 0 END) as winning_trades,
            COUNT(*) as total_trades
        FROM trades
        WHERE exit_bar IS NOT NULL
        GROUP BY strategy_id
    )
    SELECT 
        strategy_id,
        num_trades,
        ROUND(avg_return_pct, 4) as avg_return_pct,
        ROUND(total_return_pct, 2) as total_return_pct,
        ROUND(winning_trades * 100.0 / total_trades, 2) as win_rate_pct
    FROM strategy_performance
    WHERE num_trades > 10
    ORDER BY avg_return_pct DESC
    LIMIT 20
''')
print("Top 20 Strategies by Average Return:")
performance

## Signal Frequency Distribution

In [ ]:
# Distribution of signal frequencies
frequency_dist = ta.sql('''
    WITH strategy_signals AS (
        SELECT 
            strategy_id,
            COUNT(*) as signal_count
        FROM traces
        GROUP BY strategy_id
    )
    SELECT 
        CASE 
            WHEN signal_count < 10 THEN '0-9'
            WHEN signal_count < 50 THEN '10-49'
            WHEN signal_count < 100 THEN '50-99'
            WHEN signal_count < 200 THEN '100-199'
            WHEN signal_count < 500 THEN '200-499'
            ELSE '500+'
        END as signal_range,
        COUNT(*) as strategy_count
    FROM strategy_signals
    GROUP BY signal_range
    ORDER BY MIN(signal_count)
''')
print("Distribution of Signal Frequencies:")
frequency_dist

## Saved Queries for Future Use

In [5]:
# Save useful queries
queries = {
    'top_performers': '''
        WITH signal_changes AS (
            SELECT 
                strategy_id,
                bar_idx,
                signal_value,
                price,
                LAG(signal_value) OVER (PARTITION BY strategy_id ORDER BY bar_idx) as prev_signal
            FROM traces
        ),
        trades AS (
            SELECT 
                strategy_id,
                bar_idx as entry_bar,
                price as entry_price,
                signal_value as direction,
                LEAD(bar_idx) OVER (PARTITION BY strategy_id ORDER BY bar_idx) as exit_bar,
                LEAD(price) OVER (PARTITION BY strategy_id ORDER BY bar_idx) as exit_price
            FROM signal_changes
            WHERE signal_value != 0 AND (prev_signal = 0 OR prev_signal IS NULL)
        )
        SELECT 
            strategy_id,
            COUNT(*) as num_trades,
            ROUND(AVG((exit_price - entry_price) / entry_price * direction * 100), 4) as avg_return_pct,
            ROUND(SUM((exit_price - entry_price) / entry_price * direction * 100), 2) as total_return_pct
        FROM trades
        WHERE exit_bar IS NOT NULL
        GROUP BY strategy_id
        HAVING COUNT(*) > 10
        ORDER BY avg_return_pct DESC
        LIMIT 20
    ''',
    
    'filter_by_activity': '''
        SELECT 
            strategy_id,
            COUNT(*) as signal_changes
        FROM traces
        GROUP BY strategy_id
        HAVING signal_changes BETWEEN ? AND ?
        ORDER BY strategy_id
    ''',
    
    'trade_details': '''
        WITH signal_changes AS (
            SELECT 
                strategy_id,
                bar_idx,
                signal_value,
                price,
                LAG(signal_value) OVER (PARTITION BY strategy_id ORDER BY bar_idx) as prev_signal
            FROM traces
            WHERE strategy_id = ?
        )
        SELECT 
            bar_idx,
            signal_value,
            price,
            CASE 
                WHEN signal_value != 0 AND (prev_signal = 0 OR prev_signal IS NULL) THEN 'ENTRY'
                WHEN signal_value = 0 AND prev_signal != 0 THEN 'EXIT'
                ELSE 'HOLD'
            END as action
        FROM signal_changes
        ORDER BY bar_idx
    '''
}

print("Saved queries available:")
for name in queries:
    print(f"- {name}")

Saved queries available:
- top_performers
- filter_by_activity
- trade_details


## Export Best Strategies

In [4]:
# Find and export the best strategies for production
best_strategies = ta.sql('''
    WITH signal_changes AS (
        SELECT 
            strategy_id,
            bar_idx,
            signal_value,
            price,
            LAG(signal_value) OVER (PARTITION BY strategy_id ORDER BY bar_idx) as prev_signal
        FROM traces
    ),
    trades AS (
        SELECT 
            strategy_id,
            bar_idx as entry_bar,
            price as entry_price,
            signal_value as direction,
            LEAD(bar_idx) OVER (PARTITION BY strategy_id ORDER BY bar_idx) as exit_bar,
            LEAD(price) OVER (PARTITION BY strategy_id ORDER BY bar_idx) as exit_price
        FROM signal_changes
        WHERE signal_value != 0 AND (prev_signal = 0 OR prev_signal IS NULL)
    ),
    strategy_metrics AS (
        SELECT 
            strategy_id,
            COUNT(*) as num_trades,
            AVG((exit_price - entry_price) / entry_price * direction) as avg_return,
            STDDEV((exit_price - entry_price) / entry_price * direction) as return_std,
            SUM(CASE WHEN (exit_price - entry_price) * direction > 0 THEN 1 ELSE 0 END) * 100.0 / COUNT(*) as win_rate
        FROM trades
        WHERE exit_bar IS NOT NULL
        GROUP BY strategy_id
    )
    SELECT 
        strategy_id,
        num_trades,
        ROUND(avg_return * 100, 4) as avg_return_pct,
        ROUND(win_rate, 2) as win_rate_pct,
        ROUND(avg_return / NULLIF(return_std, 0) * SQRT(252 * 78), 2) as sharpe_ratio
    FROM strategy_metrics
    WHERE num_trades > 50
        AND avg_return > 0
        AND win_rate > 45
    ORDER BY sharpe_ratio DESC
    LIMIT 10
''')

print("Best strategies for production:")


NameError: name 'ta' is not defined

In [2]:
# Save to file
if len(best_strategies) > 0:
    best_strategies.to_json('best_keltner_strategies.json', orient='records', indent=2)
    print(f"\nSaved {len(best_strategies)} strategies to best_keltner_strategies.json")

NameError: name 'best_strategies' is not defined

In [ ]:
import sys
sys.path.append('../../src')
from analytics.simple_analytics import TraceAnalysis

ta = TraceAnalysis('results/20250622_180858')

# See how many strategies you have
ta.sql("SELECT COUNT(DISTINCT strategy_id) FROM traces")

# See some actual data
ta.sql("SELECT * FROM traces LIMIT 10")